In [44]:
%%capture
%run 01_transaction.ipynb

In [45]:
from eth_account.messages import encode_defunct as encode_msg
import web3 as w3; w3 = w3.Account

Create a public key `pub_key`, private key `priv_key` pair. The public key is used as an address and the private key is used for signing txs.

In [46]:
def keys():
    acc = w3.create()
    return acc.privateKey.hex(), acc.address

pub_key, priv_key = keys()
ph(pub_key), ph(priv_key)

('0xa3a8903f15a4c0...1a1', '0x67bf211542C802...c7e')

## Account

Holds a key pair and is used for signing txs.

In [47]:
class Account:
    def __init__(self): 
        self.priv, self.pub = keys()
        self.nonce = 0
            
    def sign(self, to, value):
        tx = TX(self.pub, to.pub, value, self.nonce)
        self.nonce += 1
        m = encode_msg(bytes(tx))
        sig = w3.sign_message(m, self.priv)
        tx.sig = sig
        return tx
    
    def __str__(self): return hash2emoji(self.pub)+' '+ph(self.pub)

In [48]:
acc1 = Account(); print(acc1)
acc2 = Account(); print(acc2)

🕄 0xDEF9Dd2AA8fFbA...29b
💰 0x4Aba7F3d9Eb856...Efa


### Transaction Signing
Signing a tx generates a signature that only the private key can produce. This can be later validated very efficiently.

In [61]:
signed_tx = acc1.sign(acc2, 12)
print(signed_tx)
print(signed_tx.sig)

time:	Fri Mar 26 00:53:40 2021
from:	🕄 0xDEF9Dd2AA8fFbA...29b
to:	💰 0x4Aba7F3d9Eb856...Efa
value:	12 ether
nonce:	2
signed:	true

SignedMessage(messageHash=HexBytes('0x1f5ec8be6257dbe9949ec184008e5f50de1776e725f1edf28730ecad51b2b401'), r=22143422187490374837116784073248084064396470869059143199853201972073760713121, s=18817258494161734004261010152895941288140257758356398526420532459161506354634, v=27, signature=HexBytes('0x30f4bb99c046868754d4292be2ebe5c83d3dabe44c349ef78231aa0264a269a1299a30dd74c969364a9d3ce40f723ba7fc327c3b4417aac60478ff9dc5cd15ca1b'))


### Transaction Validation

Validates if the tx was signed by the sender.

In [62]:
def validate_tx(tx):
    m = encode_msg(bytes(tx))
    return w3.recover_message(m, signature=tx.sig.signature) == tx.fr

In [63]:
assert validate_tx(signed_tx)

If anything in the tx is changed, like increasing the value to send, the tx signature should become invalid.

In [67]:
signed_tx.value = 30
assert not validate_tx(signed_tx)